In [99]:
import pyodbc 
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [45]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=mrpink;'
                      'Database=MOE;'
                      'Trusted_Connection=yes;')

In [48]:
#query_time_filter = "SELECT * FROM [ndot_atspm].[dbo].[Controller_Event_Log] WHERE SignalID='86' AND (Timestamp >= '20190710 05:30:58' AND Timestamp <= '20190710 05:35:58') ORDER BY Timestamp ASC;"
#query = "SELECT * FROM [MOE].[dbo].[Controller_Event_Log_Ndot] WHERE SignalID='86' ORDER BY Timestamp ASC;"
#df= pd.io.sql.read_sql(query, conn)

In [90]:
def print_data_within( sig_name, start_date, hour_start, end_date, hour_end):
    sig = dbo_Signals[dbo_Signals.Location==sig_name]['SignalID']
    start_date =  start_date + timedelta(hours=hour_start)
    end_date =  end_date + timedelta(hours=hour_end)
    start_date = pd.Timestamp(start_date)
    end_date = pd.Timestamp(end_date)
    start_date_string = start_date.strftime('%Y%m%d %H:00:00')
    end_date_string = end_date.strftime('%Y%m%d %H:00:00')
    query = "SELECT * FROM [MOE].[dbo].[Controller_Event_Log_Ndot] WHERE SignalID='"+str(sig.iloc[0])+"' AND (Timestamp >= '"+ start_date_string +"' AND Timestamp <=  '"+ end_date_string +"') ;"
    stat_df= pd.io.sql.read_sql(query, conn)
    total_events = stat_df['EventCode'].nunique()
    num_articles = len(stat_df)
    approach_volume(stat_df)
    
 

In [108]:
start_date = widgets.DatePicker(description='Start Date', value=pd.to_datetime('2019-07-10'))

end_date = widgets.DatePicker(description='End Date', value=pd.to_datetime('2019-07-10'))

hour_start = widgets.IntSlider(description='Start Hour', min=0, max=23,step=1)

hour_end = widgets.IntSlider(description='End Hour', min=0, max=23,step=1)

#sig_id = widgets.IntSlider(description='Signal ID', min=0, max=180,step=1)

dbo_Signals_query =  query = "SELECT * FROM [MOE].[dbo].[Signals];"
dbo_Signals = pd.io.sql.read_sql(dbo_Signals_query, conn)
dbo_Signals['Location'] = dbo_Signals.PrimaryName+" & "+dbo_Signals.SecondaryName
sig_name = widgets.RadioButtons(
    options=dbo_Signals.Location,
    description='Select Signal:',
    disabled=False
)


out = widgets.interactive_output(print_data_within, {'sig_name': sig_name, 'start_date': start_date, 'hour_start':hour_start, 'end_date': end_date, 'hour_end':hour_end})
widgets.VBox([widgets.VBox([sig_name, start_date,hour_start,end_date,hour_end]), out])

In [112]:
def approach_volume(df):
    volume_data = df[df.EventCode==82]
    dbo_Signals_query =  query = "SELECT * FROM [MOE].[dbo].[Signals];"
    dbo_Signals = pd.io.sql.read_sql(dbo_Signals_query, conn)

    df.groupby(['EventParams']).count()['SignalID'].plot()
    #print(df[:3])
    

In [115]:
import tabpy_client